<h1>Política (π):</h1>

Uma política é uma estratégia que define o comportamento do agente em um determinado ambiente.
Mais especificamente, uma política determina qual ação o agente deve tomar em cada estado para maximizar a recompensa cumulativa ao longo do tempo.




<h3>Política Determinística (𝜋):</h3>

Define uma ação específica para cada estado, é a ação que o agente deve tomar quando está no estado S.





<h3>Política Estocástica (𝜋(a∣s)):</h3>

Define uma distribuição de probabilidade sobre as ações para cada estado, é a probabilidade de o agente tomar a ação A no estado S.

<h1>As funções de valor:</h1>

Uma função de valor em aprendizagem por reforço (Reinforcement Learning, RL) tem o objetivo de estimar a "qualidade" de estados ou ações em termos de recompensas futuras esperadas.

Permitem que o agente consulte a qualidade do estado atual (S)
em vez de esperar para observar o resultado a longo prazo.
Elas calculam todos os retornos possíveis calculando a média das recompenas esperadas.


<h1>A função de valor do estado:</h1>

Calcula a recompensa total esperada que um agente pode obter a partir desse estado,
seguindo uma política específica.

É o prêmio futuro que o agente pode esperar receber a partir de um determinado estado.

Representa o valor esperado de estar em um detrminado estado.

<h1>A função de valor de ação:</h1>

Calcula a recompensa total esperada que um agente pode obter após tomar a ação
A a partir do estado S e seguir uma política específica.

Representa o valor esperado de executar uma ação A em um estado e seguir a politica

Descreve o que acontece quando o agente seleciona pela 1ª vez uma ação.

Nos permite avaliar a probabilidade de recomepensa em cada estado possível.

<h1>Função de Valor do Estado V(s):</h1>

Estima o valor de estar em um estado específico s, independentemente da ação tomada.

<h1>Função de Valor de Ação Q(s,a):</h1>

Estima o valor de tomar uma ação específica a em um estado s.

In [1]:
import numpy as np

In [2]:
# Definindo a função de recompensa
def reward(state):
    return 1 if state == 0 else 0

In [3]:
# Definindo a função de transição
def next_state(state, action):
    row, col = divmod(state, 4)
    if action == 0 and row > 0:  # Cima
        row -= 1
    elif action == 1 and row < 3:  # Baixo
        row += 1
    elif action == 2 and col > 0:  # Esquerda
        col -= 1
    elif action == 3 and col < 3:  # Direita
        col += 1
    return row * 4 + col

In [10]:
# Inicializando a política aleatória (probabilidade igual para cada ação)
policy = np.ones((16, 4)) / 4 # Política inicial uniforme

# Inicializando os valores
q_values = np.zeros(16)
size = q_values.shape[0]

In [6]:
def policy_evaluation(policy, values, discount_factor=0.9, theta=0.001):
    while True:  # Loop principal de avaliação da política
        delta = 0  # Inicializa delta para monitorar a mudança máxima nos valores de estado

        for s in range(size):  # Itera por todos os estados
            v = values[s]  # Armazena o valor antigo do estado s
            # Calcula o novo valor do estado s sob a política atual
            new_value = sum(policy[s][a] * (reward(next_state(s, a)) + discount_factor * values[next_state(s, a)]) for a in range(4))
            values[s] = new_value  # Atualiza o valor do estado s
            delta = max(delta, abs(v - new_value))  # Atualiza delta com a maior mudança observada

        # Verifica se a mudança máxima nos valores de estado é menor que o limiar theta
        if delta < theta:
            break  # Se sim, sai do loop

    return values  # Retorna os valores dos estados após a avaliação

In [7]:
def policy_improvement(policy, values, discount_factor=0.9):
    policy_stable = True  # Inicializa a variável que verifica se a política é estável
    for s in range(size):  # Itera por todos os estados
        old_action = np.argmax(policy[s])  # Obtém a ação antiga (ação com maior probabilidade)
        action_values = np.zeros(4)  # Inicializa o array para armazenar os valores das ações
        for a in range(4):  # Itera por todas as ações possíveis
            # Calcula o valor esperado da ação a a partir do estado s
            action_values[a] = reward(next_state(s, a)) + discount_factor * values[next_state(s, a)]
        best_action = np.argmax(action_values)  # Seleciona a melhor ação com base nos valores calculados
        if old_action != best_action:  # Verifica se a ação mudou
            policy_stable = False  # Se mudou, a política não é estável
        policy[s] = np.eye(4)[best_action]  # Atualiza a política para o estado s com a melhor ação
    return policy, policy_stable  # Retorna a nova política e se ela é estável

In [8]:
def policy_iteration(policy, values, discount_factor=0.9, theta=0.001):
    while True:  # Loop principal da iteração de políticas
        # Avaliação da política: calcula os valores dos estados sob a política atual
        values = policy_evaluation(policy, values, discount_factor, theta)

        # Melhoria da política: ajusta a política com base nos valores calculados
        policy, policy_stable = policy_improvement(policy, values, discount_factor)

        # Verifica se a política se estabilizou (não houve mudanças)
        if policy_stable:
            break  # Se a política é estável, sai do loop
    return policy, values  # Retorna a política e os valores ótimos

In [9]:
# Executar a iteração de políticas
optimal_policy, optimal_values = policy_iteration(policy, q_values)

print("cima: 0, baixo: 1, esquera: 2, direita: 3")
# Mostrar a política ótima e os valores ótimos
print("Política Ótima (melhores ações para cada estado):")
optimal_actions = np.argmax(optimal_policy, axis=1).reshape(4, 4)
print(optimal_actions)
print("\nValores Ótimos dos Estados:")
optimal_values = optimal_values.reshape(4, 4)
print(optimal_values)

cima: 0, baixo: 1, esquera: 2, direita: 3
Política Ótima (melhores ações para cada estado):
[[0 2 2 2]
 [0 0 0 0]
 [0 0 0 0]
 [0 0 0 0]]

Valores Ótimos dos Estados:
[[9.99233945 9.9931055  8.99379495 8.09441546]
 [9.9931055  8.99379495 8.09441546 7.28497391]
 [8.99379495 8.09441546 7.28497391 6.55647652]
 [8.09441546 7.28497391 6.55647652 5.90082887]]
